In [0]:
from pyspark.sql.functions import col, to_timestamp, to_date, date_format, current_timestamp

In [0]:
# path silver aws s3
path_silver = "s3://aws-data-lakehouse/silver/crypto/"
path_gold = "s3://aws-data-lakehouse/gold/crypto/"
# read df_silver
df_silver = spark.read.format("delta").load(path_silver)
#display df_silver
display(df_silver.limit(5))

In [0]:
df_filtered = df_silver.select("name","current_price","last_updated") \
    .withColumn("date_hour_ref",date_format(col("last_updated"), "yyyy-MM-dd HH")) \
    .withColumn("processed_to_gold", current_timestamp())

df_sorted = df_filtered.select("name","current_price","date_hour_ref").orderBy(
   # col("name").asc(), 
    col("date_hour_ref").desc()
)

df_sorted.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(path_gold)

display(df_sorted.limit(10))

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, avg, lag, round, when, max as max_
from pyspark.sql.window import Window

# ==========================================
# 1. Escolhe a crypto para prever e traz media por hora
# ==========================================

df_btc = df_sorted.filter(col("name") == "Bitcoin") \
    .groupBy("name", "date_hour_ref") \
    .agg(avg("current_price").alias("avg_price"))

# Comentário: Definimos a janela explicitamente para evitar o Warning de performance
window_btc = Window.partitionBy("name").orderBy("date_hour_ref")

df_features = df_btc.withColumn("lag_1", lag("avg_price", 1).over(window_btc)) \
                    .withColumn("lag_2", lag("avg_price", 2).over(window_btc)) \
                    .withColumn("lag_3", lag("avg_price", 3).over(window_btc)) \
                    .dropna()

# ==========================================
# 2. TREINAMENTO
# ==========================================
assembler = VectorAssembler(inputCols=["lag_1", "lag_2", "lag_3"], outputCol="features")
df_ml_input = assembler.transform(df_features)

rf_btc = RandomForestRegressor(featuresCol="features", labelCol="avg_price", numTrees=150)
model_btc = rf_btc.fit(df_ml_input)

# ==========================================
# 3. PREVISÃO DA PRÓXIMA HORA (T+1)
# ==========================================
# Pegamos o último registro real para servir de base para o futuro
last_rec = df_features.orderBy(col("date_hour_ref").desc()).limit(1)

# Preparamos as colunas para o T+1 (Deslocamos os lags)
last_btc_state = last_rec.select(
    "name",
    col("avg_price").alias("preco_atual"),
    col("avg_price").alias("lag_1"), 
    col("lag_1").alias("lag_2"),
    col("lag_1").alias("lag_3"), # Simplificação para o próximo passo
    col("date_hour").alias("ultima_referencia_s3")
)

# Comentário: REAPLICAMOS o assembler para criar a coluna 'features' necessária pelo modelo
df_forecast_input = assembler.transform(last_btc_state)
predictions = model_btc.transform(df_forecast_input)

# ==========================================
# 4. CÁLCULO DE VARIAÇÃO % E TENDÊNCIA
# ==========================================
df_final_percent = predictions.withColumn(
    "variacao_percentual", 
    round(((col("prediction") - col("preco_atual")) / col("preco_atual")) * 100, 4)
).select(
    "name",
    "ultima_referencia_s3",
    col("preco_atual").cast("decimal(18,2)"),
    col("prediction").alias("previsao_proxima_hora").cast("decimal(18,2)"),
    col("variacao_percentual"),
    when(col("variacao_percentual") > 0, "⬆️ SUBIR")
    .otherwise("⬇️ CAIR").alias("tendencia")
)


df_silver.write.format("delta") \
    .mode("append") \
    .option("overwriteSchema", "true") \
    .save(path_silver)

display(df_final_percent)